In [2]:
# 1- Cargar API KEY de PINECONE

from dotenv import load_dotenv
import os

# Cargar las variables del archivo .env
load_dotenv()

# Acceder a las claves
PINECONE_API_KEY= os.getenv("BD_KEY")



In [ ]:
# 2- Impmortar las librerias necesarias
!pip install sentence-transformers
from pinecone import Pinecone,ServerlessSpec,Index
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer





[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
# 3- Cargar los archivos PDF
!pip install pypdf
def read_doc(directory):
     file_loader=PyPDFLoader(directory)
     documents = file_loader.load()
     return documents


current_directory = os.getcwd()
dir_cv_aida = os.path.join(current_directory,'..', 'resume_Aida Benito.pdf')
doc_aida=read_doc(dir_cv_aida)

dir_cv_marcelo = os.path.join(current_directory,'..', 'CV_Pasut Marcelo.pdf')
doc_marcelo=read_doc(dir_cv_marcelo)

print (dir_cv_aida)




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


c:\Users\anben\Desktop\posgrado\LLM-genAI\initial_with agents\..\resume_Aida Benito.pdf


In [30]:



# Dividir el documento en fragmentos pequeños
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_doc_aida = text_splitter.split_documents(doc_aida)
split_doc_marcelo = text_splitter.split_documents(doc_marcelo)

# Ver los primeros fragmentos para asegurar que la división fue correcta
print(f"Primeros 2 fragmentos de doc_aida: {split_doc_aida[:2]}")
print(f"Primeros 2 fragmentos de doc_marcelo: {split_doc_marcelo[:2]}")



Primeros 2 fragmentos de doc_aida: [Document(page_content='AÍDABENITO\nanbenito@yahoo.com.ar\nQuilmes, BuenosAires, Argentina\nCell: (54911)6-497-6858-Skype: anbenito1975@gmail.com\nLinkedIn:https://www.linkedin.com/in/aida-benito/\nINFORMATIONSYSTEMSENGINEER\nExperiencedDataEngineer andBusiness IntelligenceConsultant. Possessesadvancedknowledgeof\ndatabases and various BI tools, as well as knowledge of programming languages, AWS and GCP\nEcosystems. Has strong communication skills. Passionate about Data Science, BigData, Internet of', metadata={'source': 'c:\\Users\\anben\\Desktop\\posgrado\\LLM-genAI\\initial_with agents\\..\\resume_Aida Benito.pdf', 'page': 0}), Document(page_content='Things, Artificial Intelligence, research and learning. Able to handle several projects simultaneously.\nHaspassedtheFirst CertificateinEnglishexam.\nCorecompetenciesinclude:\nFlexibility-Leadership-Teamwork-Conflict resolution-Analytical thinking-Decisionmaking\nCustomerService-Creativethinking\nPROFE

In [39]:
# 4- Inicializar Pinecone
from pinecone import ServerlessSpec
import time
pc=Pinecone(api_key=PINECONE_API_KEY)
cloud = 'aws'
region = 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)



index_name1 = "resume-index-aida"
index_name2 = "resume-index-marcelo"


def delete_if_exists(index_name,pc):
   if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
    print("index {} borrado".format(index_name))

    time.sleep(1)

delete_if_exists(index_name1,pc)
delete_if_exists(index_name2,pc)

# Si el índice no existe, lo creamos

def create_index(index_name,pc):
  if index_name not in pc.list_indexes().names():
      pc.create_index(
          index_name,
          dimension=384,  
          metric='cosine',
          spec=spec)

create_index(index_name1,pc)
create_index(index_name2,pc)

# connect to index
index_aida = pc.Index(index_name1)
index_marcelo = pc.Index(index_name2)
time.sleep(1)

# view index stats
print("index_aida: {}".format(index_aida.describe_index_stats()))
print("index_marcelo: {}".format(index_marcelo.describe_index_stats()))


index resume-index-aida borrado
index resume-index-marcelo borrado
index_aida: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
index_marcelo: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [40]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings usando el modelo 


texts_aida = [doc.page_content for doc in split_doc_aida]
texts_marcelo = [doc.page_content for doc in split_doc_marcelo]
aida_embeddings = embed_model.encode(texts_aida)
marcelo_embeddings = embed_model.encode(texts_marcelo)


def upsert_embeddings(embeddings,index_name,index,texts,name):
    upsert_data_generator = [
        {
            'id': str(i), 
            'values': embedding, 
            'metadata': {'text': texts[i], 'persona': name}
        }
        for i, embedding in enumerate(embeddings)
    ]
    index.upsert(vectors=upsert_data_generator)
    print(f"Embeddings generados y almacenados en el índice '{index_name}' de Pinecone.")

upsert_embeddings(aida_embeddings,index_name1,index_aida,texts_aida,"Aida")
upsert_embeddings(marcelo_embeddings,index_name2,index_marcelo,texts_marcelo,"Marcelo")



Embeddings generados y almacenados en el índice 'resume-index-aida' de Pinecone.
Embeddings generados y almacenados en el índice 'resume-index-marcelo' de Pinecone.
